In [1]:
#%%capture
!pip install bitsandbytes
!pip install transformers==4.34.0
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 15.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver doe

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

2024-04-16 10:24:26.883197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 10:24:26.883338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 10:24:27.012161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

In [4]:
pipe = pipeline(
    "text-generation",
    model="Jyotiyadav/copywriting",
    #tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

adapter_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Result1 for MenuDataset

In [23]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
pizza margherita 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]</s>",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
pizza margherita 

### Output:
 [/INST]</s>

"""
  "ingredients": ["tomato sauce", "mozzarella", "basil"],
  "description": "A classic Italian pizza with tomato sauce, mozzarella chee

In [26]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Pizza Napoletana 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Pizza Napoletana 

### Output:
 [/INST]

### Sample Input:
Pizza Napoletana

### Sample Output:
[
  {
    "ingredients": [
      "Tomato",
      "Mozzarella",
      "Basil"
    ],


### Result2 for MenuDataset

In [28]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Pasta carbonara  

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Pasta carbonara  

### Output:
 [/INST]

### Input data:
Pasta carbonara

### Output:
[INST]
{
  "ingredients": [
    "Pasta",
    "Bacon",
    "Egg",
    "Cheese"
  ],
  "description

### Result3 for MenuDataset

In [37]:

prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Tiramisù 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Tiramisù 

### Output:
 [/INST]

### Expected Output:

```
{
  "ingredients": [
    "Mascarpone",
    "Egg",
    "Coffee",
    "Ladyfingers",
    "Cocoa powder",
    "Sugar",
    "Bra

### Result4 for MenuDataset

In [43]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Lasagne alla Bolognese

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Lasagne alla Bolognese

### Output:
 [/INST]

### Expected Output:

{
  "ingredients": [
    "Pasta",
    "Beef",
    "Tomato",
    "Cheese",
    "Onion",
    "Garlic",
    "Olive 

### Result5 for MenuDataset

In [60]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Pasta with Gricia Sauce

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Pasta with Gricia Sauce

### Output:
 [/INST]

### Input data:
Pasta with Gricia Sauce

### Output:
 [/INST]

### Input data:
Pasta with Gricia Sauce

### Output:
 [/INST]

### Inp

### Result6 for MenuDataset

In [55]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Torta Caprese

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Torta Caprese

### Output:
 [/INST]

### Input data:
Torta Caprese

### Output:
 [/INST]

### Input data:
Torta Caprese

### Output:
 [/INST]

### Input data:
Torta Caprese

### Ou

### Result7 for MenuDataset

In [56]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Cannoli

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Cannoli

### Output:
 [/INST]

### Expected Output:

```
{
  "ingredients": [
    "Cannoli",
    "Cream",
    "Chocolate",
    "Sugar",
    "Eggs",
    "Milk",
    "Cornstarch",
  

### Result8 for MenuDataset

In [57]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Spaghetti alla puttanesca

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Spaghetti alla puttanesca

### Output:
 [/INST]

### Input data:
Spaghetti alla puttanesca

### Output:
 [/INST]

### Input data:
Spaghetti alla puttanesca

### Output:
 [/INST]

#

### Result 9

In [58]:
prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Taralli

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] As a restaurant menu manager, your role is to gather below informations based on input Data(Name of the dish).
generate the output 

### information to be extracted :
<Ingredients>: Only Ingredients included in the dish.
<Description>: Briefly describe the dish.
<Allergens>: Only Choose relevant options from this list - [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams].
<Additional Information>: Only Choose relevant options from this list - [Spicy, Vegan, Gluten free, Vegetarian].

### Output Format 
"""
  "ingredients": All Ingredients in a List,
  "description": Description in a string,
  "allergen": All allergen in a List,
  "Additional_information": All Additional_information in a List
"""

### Input data:
Taralli

### Output:
 [/INST]

### Input data:
Taralli

### Output:
 [/INST]

### Input data:
Taralli

### Output:
 [/INST]

### Input data:
Taralli

### Output:
 [/INST]

### Inpu